In [ ]:
# autoreload
%reload_ext autoreload
%autoreload 2

# Recreating the previous meta-analysis

i.e. '[Understanding coralline algal responses to ocean acidification: Meta-analysis and synthesis](https://onlinelibrary.wiley.com/doi/full/10.1111/gcb.15899)' (Cornwall et al., 2022)

The R code used for the analysis (Meta-analysis Figure 5-6.Rmd), along with the two .csv datasets (juvenile_raw_data(in).csv, juvenile_raw_data(in).csv) were provided by Ben Harvey.

The following is a translation of the original R code into Python to compare and contrast with our new analysis.
